In [1]:
import plotly, dash
import plotly.express as px
import pandas as pd
from dash import Dash, html, dash_table, dcc, Input, Output

df = pd.read_csv('gyroscope_data.csv')

app = Dash()

# Initial values
initial_samples = 100  # Default number of samples
start_index = 0  # Start position

app.layout = html.Div([
    html.H1("Gyroscope Data"),
    
    html.P("Choose a graph type:"),
    dcc.Dropdown(
        id='graph-type',
        options=[{'label': i, 'value': i} for i in ['scatter', 'line', 'bar', 'histogram', 'box', 'violin']],
        value='scatter'
    ),
    
    html.P("Choose a column:"),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': 'All (x, y, z)', 'value': 'all'}] + [{'label': col, 'value': col} for col in df.columns],
        value='x'
    ),
    
    html.P("Enter number of samples to display:"),
    dcc.Input(id='sample-size', type='number', value=initial_samples, min=10, step=10),
    
    html.Div([
        html.Button("Previous", id="prev-btn", n_clicks=0, style={'margin-right': '10px'}),
        html.Button("Next", id="next-btn", n_clicks=0),
    ], style={'margin-bottom': '10px'}),
    
    dcc.Graph(id='graph'),
    
    html.H3("Statistical Summary of Displayed Data"),
    dash_table.DataTable(id='stats-table', page_size=10)  # Dynamic table
])


@app.callback(
    Output('graph', 'figure'),
    Output('stats-table', 'data'),
    Output('stats-table', 'columns'),
    Input('graph-type', 'value'),
    Input('dropdown', 'value'),
    Input('sample-size', 'value'),
    Input('prev-btn', 'n_clicks'),
    Input('next-btn', 'n_clicks')
)

def update_graph(graph_type, column, sample_size, prev_clicks, next_clicks):
    # Calculate start and end index for navigation
    max_samples = len(df)
    start = max(0, (next_clicks - prev_clicks) * sample_size)
    end = min(start + sample_size, max_samples)

    # Select subset of data
    df_subset = df.iloc[start:end]

    # Handle "All" selection
    y_values = ['x', 'y', 'z'] if column == 'all' else [column]
    y_values = [col for col in y_values if col in df.columns]

    if not y_values:
        return {}, [], []

    # Create graph based on selection
    if graph_type in ['scatter', 'line', 'area']:
        fig = px.scatter(df_subset, x=df_subset.index, y=y_values, title=f'{graph_type.capitalize()} Plot ({start} to {end})') \
            if graph_type == 'scatter' else \
            px.line(df_subset, x=df_subset.index, y=y_values, title=f'{graph_type.capitalize()} Chart ({start} to {end})') \
            if graph_type == 'line' else \
            px.area(df_subset, x=df_subset.index, y=y_values, title=f'Area Chart ({start} to {end})')
    else:
        fig = px.bar(df_subset, x=df_subset.index, y=y_values[0], title=f'Bar Chart ({start} to {end})') \
            if graph_type == 'bar' else \
            px.histogram(df_subset, x=y_values[0], title=f'Histogram ({start} to {end})') \
            if graph_type == 'histogram' else \
            px.box(df_subset, y=y_values[0], title=f'Box Plot ({start} to {end})') \
            if graph_type == 'box' else \
            px.violin(df_subset, y=y_values[0], title=f'Violin Plot ({start} to {end})')

    # Create statistical summary table
    stats_df = df_subset[y_values].describe().reset_index()
    table_columns = [{"name": col, "id": col} for col in stats_df.columns]
    table_data = stats_df.to_dict('records')

    return fig, table_data, table_columns


if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>